# The `RealParticle` class, electrons and photons

In this notebook, we will consider electrons and photons, created using the `RealParticle` class.

In [1]:
import numpy as np
import qedtool as qtl

## Electrons

We begin by creating an electron in the rest frame, i.e. its 4-momentum is $p = (m,\mathbf{0})$. Moreover, the electron is in the spin-up state with the $z$-axis as the quantization axis:

In [2]:
m = qtl.constant("electron mass")
pmu = qtl.FourVector(m, 0, 0, 0)
electron = qtl.RealParticle.electron(1, pmu, "in")

Now we actively boost the electron in the $z$-direction with $|\boldsymbol{\beta}| = 0.7$:

In [3]:
# Boost 3-vector
beta_z = qtl.ThreeVector(0.7, 0, 0)

# Boost the electron
electron_b = qtl.boost(electron, beta_z)

In _An introduction to quantum field theory_ by M.E. Peskin and D.V. Schroeder, the boosted Dirac spinor is given by

$$\Lambda_{\frac{1}{2}} u(p) = 
\begin{pmatrix}
    \sqrt{\varepsilon_\mathbf{p} - |\mathbf{p}|} \\ 0 \\ \sqrt{\varepsilon_\mathbf{p} + |\mathbf{p}|} \\ 0
\end{pmatrix}\,,$$

where $\varepsilon_\mathbf{p} = (\Lambda p)^0$ and $|\mathbf{p}|$ is the Euclidean norm of the boosted $p$:

In [4]:
# Boosted spinor from literature
p = electron_b.four_momentum.sphericals[1]
energy = electron_b.four_momentum.sphericals[0]
psi_b_literature = np.array([np.sqrt(energy - p), 0, np.sqrt(energy + p), 0], dtype=complex)

Here we compare the numerical result with the literary result:

In [5]:
print("literary: ", psi_b_literature)
print("numerical:", electron_b.polarization.bispinor)

literary:  [0.46331732+0.j 0.        +0.j 1.10291582+0.j 0.        +0.j]
numerical: [0.46331732+0.j 0.        +0.j 1.10291582+0.j 0.        +0.j]


In the large boost limit, a spin-up Dirac spinor should become (see Peskin and Schroeder)

$$\Lambda_{\frac{1}{2}} u_\uparrow(p) = \sqrt{2\varepsilon_\mathbf{p}}
\begin{pmatrix}
    0 \\ 0 \\ 1 \\ 0
\end{pmatrix}\,,$$

which we will check here:

In [6]:
# High-speed boost 3-vector
beta_large = qtl.ThreeVector(0.9999, 0, 0)

# Boosted electron
electron_large = qtl.boost(electron, beta_large)

# Construct the literary result
energy_large = electron_large.four_momentum.sphericals[0]
psi_large_literature = np.array([0, 0, np.sqrt(2*energy_large), 0], dtype=complex)

# Comparison
print("literary: ", psi_large_literature)
print("numerical:", electron_large.polarization.bispinor)

literary:  [0.       +0.j 0.       +0.j 8.5010658+0.j 0.       +0.j]
numerical: [0.06011161+0.j 0.        +0.j 8.50085327+0.j 0.        +0.j]


Users can verify by letting $|\boldsymbol{\beta}|$ approach closer to unity, that the difference between literary and numerical results above reduces. We will now create another electron, `electron_2`, and we will consider some of its properties:

In [7]:
p_2 = 0.5
energy_2 = np.sqrt(p_2**2 + m**2)
pmu_2 = qtl.FourVector(energy_2, p_2, 0.4, 1.6)
electron_2 = qtl.RealParticle.electron(1, pmu_2, "in")

`electron_2` travels at a speed

In [8]:
velocity_2 = qtl.ThreeVector.beta(pmu_2)
print(np.sqrt(velocity_2 * velocity_2))

0.6993718406068214


A rotation of the bispinor around an arbitrary axis by an angle $2\pi$ should yield the same bispinor with a minus sign:

In [9]:
# Polarization of `electron_2`
psi_0 = electron_2.polarization

# Rotation by 2π around some arbitrary axis
angle_vector_2pi = 2 * np.pi * qtl.ThreeVector(1, 0.4, 0.3)
psi_2pi = qtl.rotation(psi_0, angle_vector_2pi)

print("original:  ", np.round(psi_0.bispinor, 4))
print("2π-rotated:", np.round(psi_2pi.bispinor, 4))

original:   [ 0.4544+0.j     -0.0027+0.0921j  1.0803+0.j     -0.0064+0.2189j]
2π-rotated: [-0.4544-0.j      0.0027-0.0921j -1.0803-0.j      0.0064-0.2189j]


Moreover, a $4\pi$-rotation (around the same arbitrary axis) must give the same Dirac spinor:

In [10]:
# Rotation by 4π around some arbitrary axis
angle_vector_4pi = 4 * np.pi * qtl.ThreeVector(1, 0.4, 0.3)
psi_4pi = qtl.rotation(psi_0, angle_vector_4pi)

print("original:  ", np.round(psi_0.bispinor, 4))
print("4π-rotated:", np.round(psi_4pi.bispinor, 4))

original:   [ 0.4544+0.j     -0.0027+0.0921j  1.0803+0.j     -0.0064+0.2189j]
4π-rotated: [ 0.4544+0.j     -0.0027+0.0921j  1.0803+0.j     -0.0064+0.2189j]


## Photons

Here we will study some properties of photons. We begin by forming a left-handed 7 MeV photon in some arbitrary direction:

In [11]:
w = 7
kmu = qtl.FourVector(w, w, 1.5, 0.3)
photon = qtl.RealParticle.photon(-1, kmu, "in")

Its 4-polarization should be orthogonal to its 4-momentum, i.e. $k\cdot\epsilon = 0$:

In [12]:
emu = photon.polarization
print(kmu * emu)

-4.440892098500626e-16j


Additionally, the photon travels at the speed of light, hence its speed equals

In [13]:
velocity_photon = qtl.ThreeVector.beta(kmu)
print(np.sqrt(velocity_photon * velocity_photon))

1.0


If we now boost the photon into some other arbitrary direction, this transversality condition should still hold. First we boost the photon:

In [14]:
# Arbitrary boost 3-vector
beta = qtl.ThreeVector(0.8, 0.6, 0.7)

# Boosted photon
photon_b = qtl.boost(photon, beta)

Retrieving the boosted 4-polarization and 4-momentum, and checking the transversality and squared 4-momentum:

In [15]:
emu_b = photon_b.polarization
kmu_b = photon_b.four_momentum
print("k . e =", kmu_b * emu_b)
print("k . k =", kmu_b * kmu_b)

k . e = (3.552713678800501e-15-3.3306690738754696e-16j)
k . k = -9.947598300641403e-14


Hence, as expected, the transversality condition also holds in the new frame, and the photon's 4-momentum is still light-like.